In [1]:
import numpy as np
import pandas as pd
import pickle
import joblib
import matplotlib.pyplot as plt
import librosa, librosa.display
from IPython.display import Audio, FileLink
from pydub import AudioSegment
import os
import gc
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MaxAbsScaler

from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras import regularizers
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.activations import softmax
from tensorflow.keras.optimizers import Adam

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
iemo_data = pd.read_csv('/kaggle/input/iemocap-transcriptions-english-french/iemocapTrans.csv')

In [3]:
iemo_data.head()

,_id,activation,dominance,emotion,end_time,start_time,titre,to_translate,translated,valence
0,625682441da7a5c1eaef3689,2.5,3.5,sad,6.0541,3.9987,Ses02M_impro02_F000,I don't want you to go.,Je ne veux pas que tu partes.,2.5
1,625682441da7a5c1eaef368a,3.0,4.0,sad,15.1000,7.0366,Ses02M_impro02_M000,"I know, I know. I don't want to go either bab...",Je sais je sais. Je ne veux pas y aller non pl...,2.0
2,625682441da7a5c1eaef368b,2.5,4.5,sad,23.3599,15.5524,Ses02M_impro02_F001,I'm going to miss you too; I don't know what ...,Tu vas me manquer aussi; Je ne sais pas ce que...,1.5
3,625682441da7a5c1eaef368c,2.5,4.0,sad,26.4151,23.5790,Ses02M_impro02_F002,I don't want to be a single mom.,Je ne veux pas être une mère célibataire.,1.5
4,625682441da7a5c1eaef368d,3.0,3.5,sad,31.4253,26.7598,Ses02M_impro02_M001,You won't be. I'll be back; I'll be back befo...,Vous ne le serez pas. Je reviendrai; Je serai ...,3.5


In [4]:
iemo_data = iemo_data[['emotion', 'titre']]
iemo_data['filepath'] = '/kaggle/input/iemocap-transcriptions-english-french/Iemocap_audio/iemocap_audio/IEMOCAP_wav/' + iemo_data['titre'] + '.wav'

In [5]:
# ravdess dataset
# emotions -> 01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised
# third part of the name

emo_dict = {
    '01': 'neu',
    '02': 'neu',
    '03': 'hap',
    '04': 'sad',
    '05': 'ang',
    '06': 'fea',
    '07': 'dis',
    '08': 'sur'
}

ravdess_base = "/kaggle/input/ravdess-emotional-speech-audio/"

rav_data = pd.DataFrame(columns=['emotion', 'titre', 'filepath'])

for dirname, _, filenames in os.walk(ravdess_base):
    for filename in filenames:
        
        info_list = filename.split('-')
        emotion = emo_dict[info_list[2]]
                
        new_row = {
            'emotion': [emotion],
            'titre': [filename[:-4]],
            'filepath': [os.path.join(dirname, filename)]
        }
        rav_data = pd.concat([rav_data, pd.DataFrame(new_row)], ignore_index=True)
rav_data.head()

,emotion,titre,filepath
0,sur,03-01-08-01-01-01-02,/kaggle/input/ravdess-emotional-speech-audio/A...
1,neu,03-01-01-01-01-01-02,/kaggle/input/ravdess-emotional-speech-audio/A...
2,dis,03-01-07-02-01-02-02,/kaggle/input/ravdess-emotional-speech-audio/A...
3,dis,03-01-07-01-01-02-02,/kaggle/input/ravdess-emotional-speech-audio/A...
4,neu,03-01-01-01-02-01-02,/kaggle/input/ravdess-emotional-speech-audio/A...


In [6]:
# tess dataset

tess_base = "/kaggle/input/toronto-emotional-speech-set-tess/TESS Toronto emotional speech set data"

tess_dict = {
    'neutral': 'neu',
    'pleasant_surprised': 'sur',
    'happy': 'hap',
    'Sad': 'sad',
    'sad': 'sad',
    'angry': 'ang',
    'Fear': 'fea',
    'fear': 'fea',
    'disgust': 'dis',
    'Pleasant_surprise': 'sur'
}

tess_data = pd.DataFrame(columns=['emotion', 'titre', 'filepath'])

for dirname, _, filenames in os.walk(tess_base):
    for filename in filenames:
        
        new_row = {
            'emotion': [tess_dict[dirname.split('/')[-1][4:]]],
            'titre': [filename[:-4]],
            'filepath': [os.path.join(dirname, filename)]
        }
        tess_data = pd.concat([tess_data, pd.DataFrame(new_row)], ignore_index=True)
        
tess_data.head()

,emotion,titre,filepath
0,fea,YAF_home_fear,/kaggle/input/toronto-emotional-speech-set-tes...
1,fea,YAF_youth_fear,/kaggle/input/toronto-emotional-speech-set-tes...
2,fea,YAF_near_fear,/kaggle/input/toronto-emotional-speech-set-tes...
3,fea,YAF_search_fear,/kaggle/input/toronto-emotional-speech-set-tes...
4,fea,YAF_pick_fear,/kaggle/input/toronto-emotional-speech-set-tes...


In [7]:
data = pd.concat([iemo_data, rav_data, tess_data], ignore_index=True)

In [8]:
data = data.sample(frac=1)

In [9]:
def show_cat(df):
    print('sad', df.emotion.loc[df.emotion == 'sad'].count())
    print('fru', df.emotion.loc[df.emotion == 'fru'].count())
    print('neu', df.emotion.loc[df.emotion == 'neu'].count())
    print('hap', df.emotion.loc[df.emotion == 'hap'].count())
    print('exc', df.emotion.loc[df.emotion == 'exc'].count())
    print('sur', df.emotion.loc[df.emotion == 'sur'].count())
    print('ang', df.emotion.loc[df.emotion == 'ang'].count())
    print('fea', df.emotion.loc[df.emotion == 'fea'].count())
    print('oth', df.emotion.loc[df.emotion == 'oth'].count())
    print('dis', df.emotion.loc[df.emotion == 'dis'].count())
    
show_cat(data)

sad 2034
fru 2917
neu 2702
hap 1440
exc 1976
sur 894
ang 2053
fea 891
oth 26
dis 786


In [10]:
data = pd.get_dummies(data, columns=['emotion'], dtype='int')
data.head()

,titre,filepath,emotion_ang,emotion_dis,emotion_exc,emotion_fea,emotion_fru,emotion_hap,emotion_neu,emotion_oth,emotion_sad,emotion_sur
4728,Ses05F_script01_1_M027,/kaggle/input/iemocap-transcriptions-english-f...,0,0,0,0,1,0,0,0,0,0
7640,Ses04M_impro08_M025,/kaggle/input/iemocap-transcriptions-english-f...,0,0,0,0,0,0,1,0,0,0
4926,Ses02M_impro06_F014,/kaggle/input/iemocap-transcriptions-english-f...,0,0,0,0,0,0,0,0,1,0
10589,03-01-04-01-01-02-15,/kaggle/input/ravdess-emotional-speech-audio/A...,0,0,0,0,0,0,0,0,1,0
9868,Ses03F_impro07_M024,/kaggle/input/iemocap-transcriptions-english-f...,0,0,1,0,0,0,0,0,0,0


In [11]:
def add_noise(data, noise_factor):
    noise = np.random.randn(len(data))
    noice_data = data + noise_factor * noise
    noice_data = noice_data.astype(type(data[0]))
    return noice_data

def change_pitch(data, sampling_rate, n_steps=3):
    return librosa.effects.pitch_shift(data, sr=sampling_rate, n_steps=n_steps)

In [12]:
def audio_to_stft(filepath, emotions, sample_rate):
    
    arr_len = 3*sample_rate
    arr, sr = librosa.load(filepath, sr=sample_rate)
    
    audios = []
    
    while (arr.shape[0] >= sample_rate):
        
        if arr.shape[0] < arr_len:
            
            arr = np.pad(arr, (0, arr_len-arr.shape[0]), 'constant')
            
#             if ((emotions['emotion_sur'] == 1) | (emotions['emotion_fea'] == 1) | (emotions['emotion_dis'] == 1)):
    #                 # create noice audio
    #                 noise_audio = add_noise(arr, 0.0001)
    #                 audios.append(librosa.amplitude_to_db(np.abs(librosa.core.stft(noise_audio, n_fft=1024, hop_length=512))))
#                 # change pitch of audio
#                 pitch_audio = change_pitch(arr, sr)
#                 audios.append(librosa.amplitude_to_db(np.abs(librosa.core.stft(pitch_audio, n_fft=1024, hop_length=512))))
            audios.append(librosa.amplitude_to_db(np.abs(librosa.core.stft(arr, n_fft=1024, hop_length=512))))
            arr = np.zeros(0)

        else:
                    
            seg = arr[:arr_len]
#             if ((emotions['emotion_sur'] == 1) | (emotions['emotion_fea'] == 1) | (emotions['emotion_dis'] == 1)):
#                 # create noice audio
#                 noise_audio = add_noise(seg, 0.0001)
#                 audios.append(librosa.amplitude_to_db(np.abs(librosa.core.stft(noise_audio, n_fft=1024, hop_length=512))))
#                 # change pitch of audio
#                 pitch_audio = change_pitch(seg, sr)
#                 audios.append(librosa.amplitude_to_db(np.abs(librosa.core.stft(pitch_audio, n_fft=1024, hop_length=512))))
            audios.append(librosa.amplitude_to_db(np.abs(librosa.core.stft(seg, n_fft=1024, hop_length=512))))
            arr = arr[arr_len:]
            
    return audios

In [13]:
X = []
y = []

for _, row in data.iterrows():

    audios = audio_to_stft(row['filepath'], row.drop(['filepath', 'titre']), 22500)
    
    for audio in audios:
        
        # adding normal audio
        X.append(audio.reshape(171, -1, 3))
        y.append(row.drop(['filepath', 'titre']))

In [14]:
del data, iemo_data, rav_data, tess_data
gc.collect()

0

In [15]:
max = 0
for x in X:
    max1 = np.max(np.abs(x))
    if max < max1:
        max = max1

In [16]:
X_np = np.array(X, dtype=np.float16) / max
y_np = np.array(y, dtype=np.float16)

In [17]:
SPEC_SHAPE = X_np.shape[1:-1]
SPEC_SHAPE

(171, 132)

In [18]:
size = X_np.shape[0]

In [19]:
X_train = np.copy(X_np[:int(size*0.9)])
y_train = np.copy(y_np[:int(size*0.9)])
X_test = np.copy(X_np[int(size*0.9):])
y_test = np.copy(y_np[int(size*0.9):])

In [20]:
del X_np
del y_np
del X
del y
gc.collect()

0

In [21]:
# Xtr, Xte, ytr, yte = train_test_split(X_np, y_np, test_size=0.2, random_state=42)

In [22]:
resnet50 = VGG19(
    include_top = False, 
    weights = 'imagenet',
    input_shape=SPEC_SHAPE + (3,),
)

# freeze layers
for layer in resnet50.layers:
    layer.trainable = False
    
x = Flatten()(resnet50.output)
pred_layer = Dense(10, activation='sigmoid')(x)

model = Model(inputs=resnet50.input, outputs=pred_layer)
model.summary()
        

80134624/80134624 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 171, 132, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 171, 132, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 171, 132, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 85, 66, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 85, 66, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 85, 66, 128)       147584    
                                                              

In [23]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=1e-3),
    metrics=['accuracy']
)

In [24]:
# early_stopper = EarlyStopping(monitor = 'val_loss', patience = 3)
# model_save = ModelCheckpoint(filepath='models/', monitor='val_losss')

In [25]:
fit_history = model.fit(X_train, y_train, epochs=6, validation_data=(X_test, y_test))

Epoch 1/6
648/648 [==============================] - 40s 50ms/step - loss: 1.6099 - accuracy: 0.3825 - val_loss: 1.3965 - val_accuracy: 0.4707
Epoch 2/6
648/648 [==============================] - 30s 46ms/step - loss: 1.4617 - accuracy: 0.4429 - val_loss: 1.3749 - val_accuracy: 0.4855
Epoch 3/6
648/648 [==============================] - 30s 46ms/step - loss: 1.4009 - accuracy: 0.4590 - val_loss: 1.3505 - val_accuracy: 0.4707
Epoch 4/6
648/648 [==============================] - 30s 46ms/step - loss: 1.3614 - accuracy: 0.4800 - val_loss: 1.3509 - val_accuracy: 0.4941
Epoch 5/6
648/648 [==============================] - 30s 46ms/step - loss: 1.3259 - accuracy: 0.4928 - val_loss: 1.3383 - val_accuracy: 0.5002
Epoch 6/6
648/648 [==============================] - 30s 46ms/step - loss: 1.2927 - accuracy: 0.5025 - val_loss: 1.3439 - val_accuracy: 0.4889


In [26]:
name = 'modelvgg19-without-noice-50-48'

In [27]:
model.save(f'{name}.h5') # 0.44 -> 0.40
joblib.dump(model, f'{name}.pkl')

['modelvgg19-without-noice-50-48.pkl']

In [28]:
FileLink(f'{name}.h5')

/kaggle/working/modelvgg19-without-noice-50-48.h5

In [29]:
FileLink(f'{name}.pkl')

/kaggle/working/modelvgg19-without-noice-50-48.pkl